In [1]:
import googlemaps
from datetime import datetime
from datetime import timedelta

import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

%matplotlib inline
sns.set(style="whitegrid", color_codes=True)

* Get GOOGLE API KEY, enabled for GOOGLE direction api
* export key as GOOGLE_API_KEY

In [2]:
api_key = os.getenv('GOOGLE_API_KEY', 'empty')
if api_key:
    gmaps = googlemaps.Client(key=api_key)

## Defaults

In [3]:
# Traffic models
traffic_model_array = ['best_guess', 'pessimistic', 'optimistic']
default_start_hour = 5 # am 
default_end_hour = 2   # am
time_granularity = 5   # min

# Methods

In [4]:
def getTimeEntry(route_tag, timestamp):
    """Get Time entry for timestamp
    
        returns time_entry dict
            dict_keys(['best_guess', 'timestamp', 'duration', 'pessimistic', 'optimistic'])
    """
    time_entry = dict()
    time_entry['timestamp'] = timestamp

    for model in traffic_model_array:
        direction = gmaps.directions(route_tag['start_point'], route_tag['end_point'], 
                                     mode="driving", traffic_model = model, 
                                     departure_time = timestamp)
   
        time_entry['duration'] = direction[0]['legs'][0]['duration']['value']
        try:
            time_entry[model] = direction[0]['legs'][0]['duration_in_traffic']['value']
        except KeyError:
            time_entry[model] = None
    return time_entry

In [5]:
def getStartDate(day_string = "", default_start_hour = default_start_hour):
    '''get starting date ("yyyy-mm-dd)
    
        if today set start date to today and to the next full x minutes
        else set start time to  
        
        return "" if date is in the pasr
    '''

    # if date is given make sure it is in the future or today 
    if day_string:
        start_date = datetime.strptime(day_string, "%Y-%m-%d")  
    
        if 	start_date.date() < datetime.today().date():
            print("Date is in past!")
            return ""
            
        elif start_date.date() != datetime.today().date():
            start_date = start_date.replace(hour=default_start_hour, minute=0) 
            return start_date

        
    # Set start date to today and to the next full x minutes
    start_date = datetime.now() 
    start_date += timedelta(minutes=time_granularity)
    start_date -= timedelta(minutes=start_date.minute % time_granularity, 
                            seconds=start_date.second, 
                            microseconds=start_date.microsecond)

    return start_date

In [6]:
def getRouteOneDirection(route_tag):
    """Get Routes in one direction
    
        return route
    """

    route = getRoute(route_tag)
    df = getDataFrame(route)
    plotRouteGraph(df, route)  
    
    return route

In [7]:
def getRouteBothDirections(route_tag):
    """Get Routes in both directions
    
        return route, route_back
    """
    
    route = getRoute(route_tag)
    df = getDataFrame(route)
    plotRouteGraph(df, route)  
    
    route_tag_back = dict()
    
    route_tag_back['route_date']  = route_tag['route_date']
    route_tag_back['start_point'] = route_tag['end_point'] 
    route_tag_back['end_point'] = route_tag['start_point'] 
    route_tag_back['route_tag'] = 'Reverse_{}'.format(route_tag['route_tag'])
    
    if '-' in route_tag['route_tag']:
        tags = route_tag['route_tag'].split('-')
        route_tag_back['route_tag'] = '{}-{}'.format(tags[1], tags[0])
     
    route_back = getRoute(route_tag_back)
    df_back = getDataFrame(route_back)
    plotRouteGraph(df_back, route_back) 
    
    return route, route_back    

In [8]:
def getRoute(route_tag):
    """Get Route for time window
    
        returns route dict
    """
      
    start_date = getStartDate(route_tag['route_date'])
    
    if start_date:
        end_date = start_date + timedelta(days=1)
        end_date = end_date.replace(hour=default_end_hour, minute=0) 

        route_tag['day'] = start_date.today().strftime('%Y-%m-%d')
        route_tag['time_list'] = []
        
        while 1:
            route_tag['time_list'].append(getTimeEntry(route_tag, start_date))
            start_date = start_date + timedelta(minutes=time_granularity)
                    
            if start_date > end_date:
                break
                
    return route_tag

In [9]:
def getDataFrame(route):
    '''Prepare DataFrame out of route
    
        return DataFrame
    '''
    
    df = pd.DataFrame.from_records(route['time_list'])

    # set index to time
    df.set_index(['timestamp'],inplace=True)

    # temp for this route
    df.interpolate(inplace=True)

    # seconds to minutes
    df['duration'] = df['duration']/60
    for item in traffic_model_array:
        df[item] = df[item]/60
    
    return df

In [10]:
def plotRouteGraph(df, route):
    """Plot route graph"""

    with sns.axes_style("ticks"):
        plt.figure(figsize=(18, 8))
        plt.subplots_adjust(hspace=0.001)

        plt.subplots_adjust(top=0.93)
    
        ax1 = plt.subplot(1, 1, 1)
        ax1.grid(True)

        fields = {'best_guess': "Best Guess",  
                  'duration': "no Traffic Info", 
                  'pessimistic': "Pessimistic", 
                  'optimistic': "Optimistc"}

        field_c = {'best_guess': "green",  
                  'duration': "brown", 
                  'pessimistic': "grey", 
                  'optimistic': "grey"}
        
        field_l = {'best_guess': "-",  
                  'duration': "-.", 
                  'pessimistic': "--", 
                  'optimistic': "--"}

        pltList = []
        for key, value in fields.items(): 
            plt1, = ax1.plot_date(x=df.index, y=df[key], label=value, color=field_c[key], fmt=field_l[key])
            if key == 'best_guess' or key == 'duration':
                pltList.append(plt1)
            
        ax1.fill_between(x=df.index ,y1=df['pessimistic'], y2=df['optimistic'], 
                         alpha=.2, facecolor='grey', interpolate=True)
    
        # Set x-axis
        plt.xlabel('Day: {}'.format(df.index[0].strftime('%Y-%m-%d')), fontsize='large')
    
        # Set y-axis
        total_min = min([df[key].min() for key in fields.keys()])
        total_max = max([df[key].max() for key in fields.keys()])
        ax1.set(ylim=(total_min*0.9, total_max*1.1))
        plt.ylabel('Duration (min)', fontsize='large')

        # Legend
        plt.legend(handles=pltList, loc=1)
 
        # -- Text
        text = "{}, {}\nStart: {}\nEnd: {}\nCreated {}".format(route['route_tag'], 
                                                               df.index[0].strftime('%Y-%m-%d'), 
                                                               route['end_point'], route['start_point'], 
                                                               route['day'])
        ax1.text(df.index[round(len(df)* 0.03)], total_max*1.02, text, ha='left', backgroundcolor="white")

        plt.show()